In [8]:
import os, pathlib
os.environ["LABEL_STUDIO_MODE"] = "http"
os.environ["LABEL_STUDIO_ROOT"] = str(pathlib.Path("../data/raw").resolve())
os.environ["LABEL_STUDIO_PREFIX"] = "http://localhost:8081/"



In [9]:
from paddleocr import PaddleOCR, TextDetection
from pathlib import Path
from PIL import Image
from urllib.parse import quote
import os
import json
import uuid

In [12]:
image = "../data/raw/2025_10_14 18_29 Office Lens.jpg"

In [13]:

model = TextDetection(model_name="PP-OCRv5_server_det")
output = model.predict(image, batch_size=1)
for res in output:
    res.print()
    res.save_to_img(save_path="./output/")
    res.save_to_json(save_path="./output/res.json")

Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/lsp/.paddlex/official_models/PP-OCRv5_server_det`.
{'res': {'input_path': '../data/raw/2025_10_14 18_29 Office Lens.jpg', 'page_index': None, 'dt_polys': array([[[ 507, 2967],
        ...,
        [ 507, 3048]],

       ...,

       [[1967,    0],
        ...,
        [1967,   85]]], dtype=int16), 'dt_scores': [0.9599240393972852, 0.7992695134289577, 0.6329533001107555, 0.9076279939538324, 0.6344320727857174, 0.8672609783445228, 0.8900721293464173, 0.8647542974751244, 0.7109516768942047, 0.8107423066259318, 0.8285057409200374, 0.8751889975026965, 0.8288574814599294, 0.8922069839498309, 0.8067582327402847, 0.837873264115583, 0.6199746409963284, 0.8481544643165515, 0.7640712800031367, 0.8963516775880304, 0.9001908962534174, 0.7225446159736468, 0.8286460573537537, 0.9118613200759041, 0.7091356070591665, 0.9356038020391273, 0.8896376613725255, 0.8421491763551705, 0.8693845282843901, 0

In [14]:
from PIL import Image

In [16]:
from_name = "lines"  # PolygonLabels name in Label Studio
to_name = "image"     # Image tag name
label = "text"        # Label value
model_version = "paddle-ppocrv5"
label_studio_mode = os.environ.get("LABEL_STUDIO_MODE", "local-files")  # local-files, http, or storage
label_studio_root = Path(os.environ.get("LABEL_STUDIO_ROOT", Path.cwd().resolve()))
label_studio_prefix = os.environ.get("LABEL_STUDIO_PREFIX", "/data/local-files/?d=")
image_path = Path(image).resolve()

def make_image_entry(path: Path) -> str:
    if label_studio_mode == "http":
        prefix = label_studio_prefix.rstrip("/") + "/"
        rel = path.resolve().relative_to(label_studio_root.resolve())
        return f"{prefix}{quote(rel.as_posix())}"
    rel = path.resolve().relative_to(label_studio_root.resolve())
    if label_studio_mode == "storage":
        return rel.as_posix()
    # default local-files handler
    if not label_studio_root.exists():
        raise FileNotFoundError(f"label_studio_root {label_studio_root} does not exist")
    return f"{label_studio_prefix}{quote(rel.as_posix())}"
image_entry = make_image_entry(image_path)

img_w, img_h = Image.open(image).size
raw = output[0]
res_dict = raw["res"] if isinstance(raw, dict) and "res" in raw else getattr(raw, "res", raw)
dt_polys = res_dict["dt_polys"] if isinstance(res_dict, dict) else getattr(res_dict, "dt_polys", [])
dt_scores = res_dict["dt_scores"] if isinstance(res_dict, dict) else getattr(res_dict, "dt_scores", [])
polygons = []
for poly, score in zip(dt_polys.tolist(), dt_scores):
    normalized = [
        [round(x / img_w * 100, 4), round(y / img_h * 100, 4)]
        for x, y in poly
    ]
    polygons.append({
        "id": str(uuid.uuid4()),
        "from_name": from_name,
        "to_name": to_name,
        "type": "polygonlabels",
        "score": float(score),
        "value": {
            "points": normalized,
            "polygonlabels": [label]
        }
    })

payload = [{
    "data": {"image": image_entry},
    "predictions": [{
        "model_version": model_version,
        "result": polygons,
        "score": float(sum(dt_scores) / len(dt_scores)) if dt_scores else None
    }]
}]

out_path = Path("output/ls_preannotations.json")
out_path.parent.mkdir(exist_ok=True)
out_path.write_text(json.dumps(payload, indent=2), encoding="utf-8")
print(f"Wrote {len(polygons)} polygons to {out_path}")

Wrote 36 polygons to output/ls_preannotations.json


### Importing into Label Studio
1. Make sure the labeling config contains `<Image name="image" value="$image"/>` and `<PolygonLabels name="lines" toName="image">` with a `text` label (or change the variables in the conversion cell).
2. Optional: set `LABEL_STUDIO_MODE` (`local-files` for `/data/local-files/?d=`, `http` for an external static server, or `storage` when you use Label Studio's Storage sync), along with `LABEL_STUDIO_ROOT` and `LABEL_STUDIO_PREFIX` to match your server setup. Defaults: current working directory + `/data/local-files/?d=`.
   - To mimic the LayoutLMv3 workflow, run `python serve_local_files.py --directory ../data/raw --port 8080` from the repo root, then set `LABEL_STUDIO_MODE="http"`, `LABEL_STUDIO_ROOT` to the same folder, and `LABEL_STUDIO_PREFIX="http://localhost:8080/"` before exporting.
3. Run the conversion cell; it produces `output/ls_preannotations.json` containing Paddle predictions plus the correct file URLs.
4. In your local Label Studio project choose **Import** → upload `ls_preannotations.json`. The polygons appear as grey preannotations that you can refine instead of drawing manually.